In [2]:
from osgeo import gdal, ogr

In [11]:
input_raster = gdal.Open(r"data\original\bathymetry_hawaii.tif")
type(input_raster)

osgeo.gdal.Dataset

In [13]:
# Projection
input_raster.GetProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [14]:
# Dimensions
input_raster.RasterXSize
input_raster.RasterYSize

2312

In [15]:
# Number of bands
input_raster.RasterCount

1

In [16]:
# Metadata for the raster dataset
input_raster.GetMetadata()

{'AREA_OR_POINT': 'Area',
 'elevation#grid_mapping': 'crs',
 'elevation#long_name': 'Elevation relative to sea level',
 'elevation#sdn_parameter_name': 'Sea floor height (above mean sea level) {bathymetric height}',
 'elevation#sdn_parameter_urn': 'SDN:P01::ALATZZ01',
 'elevation#sdn_uom_name': 'Metres',
 'elevation#sdn_uom_urn': 'SDN:P06::ULAA',
 'elevation#standard_name': 'height_above_mean_sea_level',
 'elevation#units': 'm',
 'lat#axis': 'Y',
 'lat#long_name': 'latitude',
 'lat#sdn_parameter_name': 'Latitude north',
 'lat#sdn_parameter_urn': 'SDN:P01::ALATZZ01',
 'lat#sdn_uom_name': 'Degrees north',
 'lat#sdn_uom_urn': 'SDN:P06::DEGN',
 'lat#standard_name': 'latitude',
 'lat#units': 'degrees_north',
 'lon#axis': 'X',
 'lon#long_name': 'longitude',
 'lon#sdn_parameter_name': 'Longitude east',
 'lon#sdn_parameter_urn': 'SDN:P01::ALONZZ01',
 'lon#sdn_uom_name': 'Degrees east',
 'lon#sdn_uom_urn': 'SDN:P06::DEGE',
 'lon#standard_name': 'longitude',
 'lon#units': 'degrees_east',
 'NC_GL

## Shapefile

In [18]:
import geopandas
input_shape = "https://opendata.arcgis.com/datasets/045b1d5147634e2380566668e04094c6_3.geojson"
gdf = geopandas.read_file(input_shape)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 76: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "D:\Programas\anaconda3\envs\gps_project\lib\site-packages\fiona\env.py", line 289, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 76: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 76: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "D:\Programas\anaconda3\envs\gps_project\lib\site-packages\fiona\env.py", line 289, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 76: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 81: invalid continuation byte

...

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "D:\Programas\anaconda3\envs\gps_project\lib\site-packages\fiona\env.py", line 289, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 72: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 71: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "D:\Programas\anaconda3\envs\gps_project\lib\site-packages\fiona\env.py", line 289, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 71: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 71: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "D:\Programas\anaconda3\envs\gps_project\lib\site-packages\fiona\env.py", line 289, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 71: invalid continuation byte
